# Quickstart

To best understand the agent framework, let's build an agent that has two tools: one to look things up online, and one to look up specific data that we've loaded into a index.

This will assume knowledge of [LLMs](../model_io) and [retrieval](../data_connection) so if you haven't already explored those sections, it is recommended you do so.

## Setup: LangSmith

By definition, agents take a self-determined, input-dependent sequence of steps before returning a user-facing output. This makes debugging these systems particularly tricky, and observability particularly important. [LangSmith](/docs/langsmith) is especially useful for such cases.

When building with LangChain, all steps will automatically be traced in LangSmith.
To set up LangSmith we just need set the following environment variables:

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="<your-api-key>"
```

## Define tools

We first need to create the tools we want to use. We will use two tools: [Tavily](/docs/integrations/tools/tavily_search) (to search online) and then a retriever over a local index we will create

### [Tavily](/docs/integrations/tools/tavily_search)

We have a built-in tool in LangChain to easily use Tavily search engine as tool.
Note that this requires an API key - they have a free tier, but if you don't have one or don't want to create one, you can always ignore this step.

Once you create your API key, you will need to export that as:

```bash
export TAVILY_API_KEY="..."
```

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
search = TavilySearchResults()

In [3]:
search.run("what is the weather in SF")

[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US',
  'content': 'recents Specialty Forecasts 10 Day Weather-San Francisco, CA Today Mon 18 | Day  Fri 22 Fri 22 | Day Foggy early, then partly cloudy later in the day. High around 60F. Winds W at 10 to 15 mph.  High 59F. Winds SSW at 10 to 15 mph. Chance of rain 60%.  Considerable cloudiness with occasional rain showers. High 59F. Winds SSE at 5 to 10 mph. Chance of rain 50%.San Francisco, CA 10-Day Weather Forecast - The Weather Channel | Weather.com 10 Day Weather - San Francisco, CA As of 12:09 pm PST Today 60°/ 54° 23% Tue 19 | Day 60° 23% S 12 mph More...'}]

### Retriever

We will also create a retriever over some data of our own. For a deeper explanation of each step here, see [this section](/docs/modules/data_connection/)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [5]:
retriever.get_relevant_documents("how to upload a dataset")[0]

Document(page_content="dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we‚Äôr

Now that we have populated our index that we will do doing retrieval over, we can easily turn it into a tool (the format needed for an agent to properly use it)

In [6]:
from langchain.tools.retriever import create_retriever_tool

In [7]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### Tools

Now that we have created both, we can create a list of tools that we will use downstream.

In [8]:
tools = [search, retriever_tool]

## Create the agent

Now that we have defined the tools, we can create the agent. We will be using an OpenAI Functions agent - for more information on this type of agent, as well as other options, see [this guide](./agent_types)

First, we choose the LLM we want to be guiding the agent.

In [9]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Next, we choose the prompt we want to use to guide the agent.

If you want to see the contents of this prompt and have access to LangSmith, you can go to:

https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In [10]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

Now, we can initalize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). For more information about how to think about these components, see our [conceptual guide](./concepts)

In [11]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools). For more information about how to think about these components, see our [conceptual guide](./concepts)

In [12]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run the agent

We can now run the agent on a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions).

In [13]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [33]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith testing'}`


[Document(page_content='LangSmith Overview and User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}), Document(page_content='Skip to main content\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. **Tracing**: LangSmith provides tracing capabilities that allow you to track the total token usage for a chain and the token usage of each step. This makes it easy to identify potentially costly parts of the chain during testing.\n\n2. **Collaborative debugging**: LangSmith simplifies the process of sharing a faulty chain with a colleague for debugging. It has a "Share" button that makes the chain and language model runs accessible to anyone with the shared link, making collaboration and debugging more efficient.\n\n3. **Collecting examples**: When testing LLM (Language Model) applications, failures and unexpected outcomes are valuable data points. LangSmith helps in identifying how a chain can fail and monitoring these failures. By testing future chain versions against known issues, you can improve the reliability and performance of your application.\n\n4. **Editing exa

In [34]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': 'recents Specialty Forecasts 10 Day Weather-San Francisco, CA Today Mon 18 | Day  Fri 22 Fri 22 | Day Foggy early, then partly cloudy later in the day. High around 60F. Winds W at 10 to 15 mph.  Considerable cloudiness with occasional rain showers. High 59F. Winds SSE at 5 to 10 mph. Chance of rain 50%.  Thu 28 | Night Cloudy with showers. Low 46F. Winds S at 5 to 10 mph. Chance of rain 40%. Fri 29 Fri 29 | DaySan Francisco, CA 10-Day Weather Forecast - The Weather Channel | Weather.com 10 Day Weather - San Francisco, CA As of 12:09 pm PST Today 60°/ 54° 23% Tue 19 | Day 60° 23% S 12 mph...'}, {'url': 'https://www.sfchronicle.com/weather/article/us-forecast-18572409.php', 'content': 'San Diego, CA;64;53;65;48;Mist in the morning;N;6;72%;41%;3 San Francisco, CA;58;45;56;43

{'input': 'whats the weather in sf?',
 'output': 'The weather in San Francisco is currently partly cloudy with a high around 60°F. The wind is coming from the west at 10 to 15 mph. There is a chance of rain showers later in the day. The temperature is expected to drop to a low of 46°F tonight with cloudy skies and showers.'}

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in previous `chat_history`. Note: it needs to be called `chat_history` because of the prompt we are using. If we use a different prompt, we could change the variable name

In [14]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

In [16]:
agent_executor.invoke(
    {
        "input": "what's my name?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

If we want to keep track of these messages automatically, we can wrap this in a RunnableWithMessageHistory. For more information on how to use this, see [this guide](/docs/expression_language/how_to/message_history)

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [21]:
message_history = ChatMessageHistory()

In [24]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [27]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent. Agents are a complex topic, and there's lot to learn! Head back to the [main agent page](./) to find more resources on conceptual guides, different types of agents, how to create custom tools, and more!